In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import geopandas as gpd 
import folium
from folium import Choropleth, Circle, Marker
from folium.plugins import HeatMap, MarkerCluster

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))


/kaggle/input/hampstead-ponds-starter/temperatures.csv
/kaggle/input/hampstead-ponds-starter/London_Lidos.csv
/kaggle/input/hampstead-ponds-starter/water_temp.csv
/kaggle/input/hampstead-ponds-starter/ponds.csv


Loading in the data from the dataset and previewing the first five rows.

In [2]:
# Read in the data
full_data = gpd.read_file("../input/hampstead-ponds-starter/London_Lidos.csv")

# View the first five rows of the data
full_data.head()

,Name,Latitude,Longitude,Website,Status,Category,Length_metres,Heated,OpeningDate,ClosingDate,Description,Source,Photo01,Photo02,geometry
0,London Fields Lido,-0.06151,51.542319,http://www.better.org.uk/leisure/london-fields...,Open,Lido,50,Yes,,,,https://www.google.com/maps/d/u/0/viewer?mid=1...,,,None
1,Tooting Bec Lido,-0.13915,51.431996,https://www.placesleisure.org/centres/tooting-...,Closed for renovations,Lido,91,No,,,,https://www.google.com/maps/d/u/0/viewer?mid=1...,,,None
2,Hampton Pool,-0.35805,51.41841,https://www.hamptonpool.co.uk/,Open,Lido,36,Yes,,,,https://www.google.com/maps/d/u/0/viewer?mid=1...,,,None
3,The Serpentine Lido,-0.168463,51.504824,https://serpentineswimmingclub.com/,Open,Open Water,100,No,,,"The Serpentine Lido is, arguably, London's mos...",https://www.google.com/maps/d/u/0/viewer?mid=1...,,,None
4,Brockwell Lido,-0.10625,51.453271,https://www.fusion-lifestyle.com/centres/brock...,Open,Lido,50,No,,,,https://www.google.com/maps/d/u/0/viewer?mid=1...,,,None


How many of each type of pool are there?

In [3]:
full_data.Category.value_counts()

Category
Lido             26
Man Made Lake     6
Open Water        4
Natural Lake      1
Name: count, dtype: int64

In [4]:
# Convert the DataFrame to a GeoDataFrame
lidos = gpd.GeoDataFrame(full_data, geometry=gpd.points_from_xy(full_data.Longitude, full_data.Latitude))

# Set the coordinate reference system (CRS) to EPSG 4326
lidos.crs = {'init': 'epsg:4326'}

# View the first five rows of the GeoDataFrame
lidos.head()

/opt/conda/lib/python3.10/site-packages/pyproj/crs/crs.py:141: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)


,Name,Latitude,Longitude,Website,Status,Category,Length_metres,Heated,OpeningDate,ClosingDate,Description,Source,Photo01,Photo02,geometry
0,London Fields Lido,-0.06151,51.542319,http://www.better.org.uk/leisure/london-fields...,Open,Lido,50,Yes,,,,https://www.google.com/maps/d/u/0/viewer?mid=1...,,,POINT (51.54232 -0.06151)
1,Tooting Bec Lido,-0.13915,51.431996,https://www.placesleisure.org/centres/tooting-...,Closed for renovations,Lido,91,No,,,,https://www.google.com/maps/d/u/0/viewer?mid=1...,,,POINT (51.43200 -0.13915)
2,Hampton Pool,-0.35805,51.41841,https://www.hamptonpool.co.uk/,Open,Lido,36,Yes,,,,https://www.google.com/maps/d/u/0/viewer?mid=1...,,,POINT (51.41841 -0.35805)
3,The Serpentine Lido,-0.168463,51.504824,https://serpentineswimmingclub.com/,Open,Open Water,100,No,,,"The Serpentine Lido is, arguably, London's mos...",https://www.google.com/maps/d/u/0/viewer?mid=1...,,,POINT (51.50482 -0.16846)
4,Brockwell Lido,-0.10625,51.453271,https://www.fusion-lifestyle.com/centres/brock...,Open,Lido,50,No,,,,https://www.google.com/maps/d/u/0/viewer?mid=1...,,,POINT (51.45327 -0.10625)


First map

In [5]:
# Create a map
m_1 = folium.Map(location=[51.5074,-0.1272], tiles='openstreetmap', zoom_start=10)

# Add points to the map
for idx, row in lidos.iterrows():
    Marker([row['Longitude'], row['Latitude']]).add_to(m_1)

# Display the map
m_1